In [2]:
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import pandas as pd
import cartopy.crs as ccrs
# from metpy.units import units
from tqdm import tqdm
path = "/glade/work/alancelin/EVT_rossbypalooza/"
os.chdir(path)
from src.utils import compute_A

In [10]:
# Combine all files in path 
# select var 
var = 'tas'
sim_number = 0
path = f"/glade/derecho/scratch/awikner/PLASIM/data/2000_year_sims_new/sim{sim_number}/{var}/"
# select lat, lon and window size
spatial_window_size = 2
# temporal_window_size = 7
reduce = 'max'
# Select lat and lon of Chicago
lat = 41.881832
lon = -87.623177 % 360 # for longitudes take the value modulo 360


files = [f for f in os.listdir(path) if f.endswith('gaussian.nc')]
files = files[:]
# take file only if year >100
files = [f for f in files if int(f.split('_')[0])>10]
# order the files by year
files = sorted(files, key=lambda x: int(x.split('_')[0]))

# disable the printing of the warning
# # combine all files using compute_A as a preprocessing function 
A = xr.open_mfdataset([path+file for file in files], preprocess=lambda ds: compute_A(ds, lat, lon, spatial_window_size, reduce), combine='nested',
                       concat_dim='time', parallel=True, decode_times=True, use_cftime=True)

A_df = A.to_dataframe()
# save the dataframe
file_name = f'data/train/A_{var}_lat.{int(lat)}_lon.{int(lon)}_spatial.{spatial_window_size}_reduce.{reduce}.csv'
if os.path.exists(file_name):
    os.remove(file_name)
A_df.to_csv(file_name)

In [5]:
# # convert index to pandas.DatetimeIndex
# A_df = A.to_dataframe()

# # select month 6, 7 or 8
# A_df = A_df[np.isin(A_df.index.month, [6, 7, 8])]
# A_df

,tas
time,
1429-06-01 00:00:00,293.131775
1429-06-02 00:00:00,293.159851
1429-06-03 00:00:00,293.405670
1429-06-04 00:00:00,293.829926
1429-06-05 00:00:00,294.181671
...,...
1069-08-27 00:00:00,309.920990
1069-08-28 00:00:00,309.836975
1069-08-29 00:00:00,309.890533


In [67]:
path = "/glade/work/alancelin/EVT_rossbypalooza/data/A_tas_41_272_2_7.csv"
A_df = pd.read_csv(path, index_col=0)
print(f"Memory usage of A_df: {A_df.memory_usage().sum() / 1024**2} MB")

Memory usage of A_df: 0.051605224609375 MB
